## Project 2: Recommendation Engines

In [6]:
import pandas as pd

In [8]:
import matplotlib 
import matplotlib.pyplot as plt
%matplotlib inline

### 1. The dataset

The movie-lens dataset 

In [9]:
path="C:/Users/talha/Documents/Training/CUNY Classes/Data612/Week1/ml-latest-small/"

In [11]:
ratings=pd.read_csv(path+'ratings.csv')

In [12]:
#ratings by user and m
ratings.head(n=5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### 2. Collaborative Filtering

#### 2.1 User - User Collaborative Filtering

#### 2.2 Item - Item Collaborative Filtering

#### 2.3 Expanding Item - Item Filtering

### 3. Evaluating the Recommendations